In [1]:
# Load all excel file
import pandas as pd

odoo_db = pd.read_csv("/Applications/MAMP/htdocs/tendercuts-site/asserts/odoo/Final_Product_template.csv")
odoo_db = odoo_db.set_index("Internal Reference")

sheet_db = pd.read_csv("/Applications/MAMP/htdocs/tendercuts-site/asserts/export_product.csv")
sheet_db = sheet_db.set_index("sku")

IOError: File /Applications/MAMP/htdocs/tendercuts-site/asserts/odoo/Final_Product_template.csv does not exist

In [119]:
import MySQLdb
import pandas as pd


class DB(object):
    def __init__(self, test=False):
        # Open database connection
#         self.db = MySQLdb.connect(
#                 "localhost",
#                 "root",
#                 "root",
#                 "dbmaster")
        self.db = MySQLdb.connect(
                "127.0.0.1",
                "tcuser",
                "oochahwielai9mahDah3",
                "v2",
                port=3307)
#         self.db = MySQLdb.connect(
#                 "127.0.0.1",
#                 "root",
#                 "!qazmlp)5",
#                 "v3_1",
#                 port=3307)
        

    def to_table(self, sql):
        cursor = self.db.cursor()

        try:
           # Execute the SQL command
           cursor.execute(sql)
           df = pd.read_sql(sql,  con=self.db)
        except Exception as e:
           print "Error: unable to fecth data"
           print (e)

        # disconnect from server

        return df

    def close(self):
        self.db.close()


dbb = DB()
mage_master = dbb.to_table("""select * from catalog_product_entity""")
mage_master = mage_master.set_index("sku")


In [6]:
# Product Sync
import pandas as pd
import functools
import xmlrpclib
products = None
class OdooConnector():
    def __init__(self):
        username = 'varun@tendercuts.in' #the user
        pwd = 'qwerty123'      #the password of the user
        dbname = 'tcuts'    #the database

        # Get the uid
        sock_common = xmlrpclib.ServerProxy ('http://localhost:8069/xmlrpc/common')
        uid = sock_common.login(dbname, username, pwd)

        # replace localhost with the address of the server
        self.sock = xmlrpclib.ServerProxy('http://localhost:8069/xmlrpc/object')
        self.execute = functools.partial(self.sock.execute_kw, dbname, uid, pwd)
    
class OdooProductLoader:
    SKIP_NAMES = [
        'Down payment',
        'Free delivery charges',
        'Express delivery charges',
        'Scheduled delivery charges',
        'Expenses',
        'Carry Bag'
    ]

    def __init__(self, connector):
        self.connector = connector
    
    def _sanitize_df(self, product_df):
        product_df = product_df[~product_df.name.isin(self.SKIP_NAMES)]
        return product_df
    
    def fetch_products(self):
        products = self.connector.execute('product.template', 'search_read', [[("active", "=", True)]], {'fields': ['default_code', 'name']})
        df = pd.DataFrame.from_records(products)
        df = df.set_index('default_code')
        return self._sanitize_df(df)

class MageSheetProductLoader:
    def __init__(self):
        pass

    def _sanitize_df(self, product_df):
        """Chuck all dep"""
        product_df = product_df[~product_df.index.str.contains("_DEP")]
        product_df = product_df[~product_df.index.str.contains("webpos_")]
        return product_df
    
    def fetch_products(self):
        sheet_db = pd.read_csv("/Applications/MAMP/htdocs/tendercuts-site/asserts/export_product.csv")
        sheet_db = sheet_db.set_index("sku")
        
        return self._sanitize_df(sheet_db)


In [19]:
odoo_loader = OdooProductLoader(OdooConnector())
mage_loader = MageSheetProductLoader()

class ProductDiff(object):
    """TODO: Need to generalize"""
    def __init__(self, odoo_loader, mage_loader):
        self.odoo_loader = odoo_loader
        self.mage_loader = mage_loader
        self.merged_df = self._merge_df(
            self.odoo_loader.fetch_products(),
            self.mage_loader.fetch_products()
        )
    
    def _merge_df(self, left_df, right_df):
        return left_df.merge(
            right_df,
            how="outer",
            left_index=True,
            right_index=True,
            suffixes=("_odoo", "_mage"))
    
    def present_in_left(self):
        return self.merged_df[self.merged_df.name_mage.isnull()]
    
    def present_in_right(self):
        return self.merged_df[self.merged_df.name_odoo.isnull()]

class OdooSync(object):
    def __init__(self, connector): #odoo_loader, mage_loader):
#         self.diff = ProductDiff(odoo_loader, mage_loader)
        self.connector = connector
    
    def delete_products(self):
        """Never to be run after initial sync"""
        #odoo_only_df = self.diff.present_in_left()
        sku = 'CHK_BIRY_CUT'
        self.connector.execute('product.product', 'unlink', [[705]])
    
    def update_skus(self, df):
        for sku, row in df.iterrows():
            if "_WHOLE" not in sku:
                continue
            try:
                self.connector.execute(
                    'product.template',
                    'write',
                    [int(row.id), {'default_code': sku.strip().replace("_WHOLE", "_SLICED")}] 
                )
            except:
                print (sku, "Failed")
    
    def mark_inactive(self, df):
        for sku, row in df.iterrows():
            if "_DEP" not in sku:
                continue
            try:
                self.connector.execute(
                    'product.template',
                    'write',
                    [int(row.id), {'active': False}] 
                )
            except:
                print (sku, "Failed")
    
    def create_product(self):

        self.connector.execute('product.template', 'create', [{
            'default_code': "MRT_MARINA_ROAST",
            'name': 'Marina Fish Roast (Seer Fish Slices)'
        }])
    
    def remove_taxes(self):
#         all_ids = self.connector.execute(
#             'product.template',
#             'search_read',
#             [[("active", "=", True)]]
#         )
        products = self.connector.execute(
            'product.template',
            'search', 
            [[("active", "=", True)]],
            {})
        print(products)
        for pid in products:
            self.connector.execute('product.template', 'write', [pid, {
                'taxes_id': 0, #([5, _, _]),
                'supplier_taxes_id': ([5, _, _])
            }])
            
        
        


syncer = OdooSync(OdooConnector())#, odoo_loader, mage_loader)

#syncer.delete_products()
# xdf = syncer.diff.present_in_right()
# syncer.update_skus(xdf)
# syncer.mark_inactive(xdf)
syncer.remove_taxes()


[135, 141, 186, 187, 143, 144, 59, 104, 198, 145, 146, 105, 39, 79, 80, 41, 81, 6, 7, 8, 61, 19, 20, 13, 64, 65, 15, 66, 67, 16, 42, 82, 18, 14, 71, 21, 43, 83, 44, 84, 45, 85, 164, 165, 12, 11, 9, 10, 62, 63, 46, 86, 148, 108, 149, 150, 27, 28, 1, 22, 23, 24, 73, 26, 31, 34, 77, 29, 75, 151, 109, 47, 92, 99, 33, 152, 153, 154, 155, 156, 157, 158, 159, 112, 169, 160, 161, 124, 142, 172, 168, 117, 36, 37, 38, 167, 40, 52, 91, 170, 118, 171, 50, 89, 90, 51, 17, 68, 147, 53, 93, 25, 74, 30, 76, 78, 35, 106, 140, 102, 101, 54, 94, 173, 119, 103, 107, 110, 72, 111, 113, 114, 115, 121, 122, 130, 133, 134, 139, 116, 120, 132, 137, 175, 32, 176, 177, 178, 123, 179, 125, 126, 56, 96, 180, 181, 127, 162, 163, 183, 129, 185, 184, 131, 100, 57, 97, 166, 188, 189, 58, 98, 48, 87, 49, 88, 191, 55, 95, 192, 136, 174, 194, 193, 60, 195, 196, 182, 128, 190, 197, 138, 69, 70]


In [105]:

merged_db = mage_master.merge(sheet_db, how="outer", left_index=True, right_index=True)

#odoo_only = merged_db[merged_db.name.isnull()]
sheet_only = merged_db[merged_db.entity_id.isnull()]
sheet_only.index
sheet_db

,type,attribute_set,parent_sku,gramsperunit,gross,net,name,sdesc,status,visibility,webpos_visible,uom,store,is_qty_decimal
sku,,,,,,,,,,,,,,
RM_CHK_WHOLE_SKIN_OFF,simple,Default,NaN,NaN,NaN,NaN,Whole Chicken With Out Skin- Raw Mat,0.900 to 1100 Gms,2.0,1,1.0,NaN,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
CHK_BIRI_CUT,simple,Default,RM_CHK_WHOLE_SKIN_OFF,550.0,500,500,Chicken Biryani Cut - Skinless,5 - 6 Pieces,1.0,4,1.0,Gms,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
CHK_WHL_SKIN_OFF,simple,Default,RM_CHK_WHOLE_SKIN_OFF,550.0,500,500,Chicken Curry Cut (Skin Off),10 - 12 Pieces,1.0,4,1.0,Gms,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
RM_CHK_MINCE,simple,Default,NaN,NaN,NaN,NaN,Minced Chicken - Raw Mat,Whole chicken deboned minced,2.0,1,1.0,NaN,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
CHK_MINCE,simple,Default,RM_CHK_MINCE,550.0,500,500,Minced Chicken,Whole chicken deboned minced,1.0,4,1.0,Gms,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
CHK_SELECT,simple,Default,RM_CHK_WHOLE_SKIN_OFF,550.0,500,500,Chicken Select,10 - 12 Pieces,1.0,4,1.0,Gms,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
RM_CHK_WHOLE_SKIN_ON,simple,Default,NaN,NaN,NaN,NaN,Whole Chicken With Skin- Raw Mat,0.900 to 1100 Gms,2.0,1,1.0,NaN,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
CHK_WHL_SKIN_ON,simple,Default,RM_CHK_WHOLE_SKIN_ON,550.0,500,500,Chicken Curry Cut (Skin On),10 - 12 Pieces,1.0,4,1.0,Gms,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1
RM_CHK_BR_BONELESS,simple,Default,NaN,NaN,NaN,NaN,Chicken Breast Bonless- Raw Mat,NaN,2.0,1,1.0,NaN,"admin,thoraipakkam,adayar,porur,tambaram,mogap...",1


In [131]:
from django.db import connections
from app.core.models import CatalogProductEntity
pd.DataFrame.from_records(list(CatalogProductEntity.objects.values()))

,created_at,entity_id,has_options,required_options,sku,type_id,updated_at,vtiger_id,vtiger_type
0,2015-11-13 07:47:41+00:00,33,0,0,webpos-chicken,simple,2017-09-18 17:08:08+00:00,NaN,None
1,2015-11-13 06:23:25+00:00,34,0,0,webpos-mutton-sheep,simple,2017-09-18 17:08:08+00:00,NaN,None
2,2015-11-13 06:55:27+00:00,35,0,0,webpos-p-Goat,simple,2017-09-18 17:08:08+00:00,NaN,None
3,2015-11-13 07:56:42+00:00,36,0,0,webpos-countrychicken,simple,2017-09-18 17:08:08+00:00,NaN,None
4,2016-06-09 16:00:58+00:00,191,0,0,CHK_NSNG_DEP,simple,2017-09-18 17:08:06+00:00,148244.0,Products
5,2016-03-20 01:36:24+00:00,192,0,0,CHK_WHL_SKIN_ON,simple,2017-09-27 09:55:13+00:00,148246.0,Products
6,2015-02-07 03:10:37+00:00,193,0,0,CHK_WHL_SKIN_OFF,simple,2017-09-27 09:55:13+00:00,148248.0,Products
7,2016-06-09 09:14:52+00:00,194,1,0,CHK_CNTRY,simple,2017-09-18 17:08:00+00:00,148250.0,Products
8,2015-09-17 05:20:21+00:00,195,0,0,CHK_BONELESS,simple,2017-09-27 09:55:13+00:00,148252.0,Products
9,2016-04-26 09:26:30+00:00,196,0,0,CHK_LEG_SKIN_OFF,simple,2017-09-27 09:55:13+00:00,148254.0,Products


In [283]:
import numpy as np
# generate initial sheet
sheet_db = pd.read_csv("/Applications/MAMP/htdocs/tendercuts-site/asserts/odoo/product.template.initial.csv")
sheet_db = sheet_db.set_index("default_code")

# external id
sheet_db['id'] = "tcuts.product_" + sheet_db.index

def foo(row):
    sku = row['default_code']
    is_raw = True
    if sku.startswith("CHK_"):
        row['categ_id/id'] = "tcuts.product_category_chicken"
        is_raw = False
    elif sku.startswith("GT_"):
        row['categ_id/id'] = "tcuts.product_category_goat"
        is_raw = False
    elif sku.startswith("LAMB_"):
        row['categ_id/id'] = "tcuts.product_category_goat"
        is_raw = False
    elif sku.startswith("SF_"):
        row['categ_id/id'] = "tcuts.product_category_seafood"
        is_raw = False
    elif sku.startswith("MRT_"):
        row['categ_id/id'] = "tcuts.product_category_marinades"
        is_raw = False
    
    elif sku.startswith("RM_CHK_"):
        row['categ_id/id'] = "tcuts.product_category_chicken"
    elif sku.startswith("RM_GT") or sku.startswith("RM_MUT"):
        row['categ_id/id'] = "tcuts.product_category_goat"
    elif sku.startswith("RM_LAMB_"):
        row['categ_id/id'] = "tcuts.product_category_goat"
    elif sku.startswith("RM_SF_"):
        row['categ_id/id'] = "tcuts.product_category_seafood"
    elif sku.startswith("RM_MRT_"):
        row['categ_id/id'] = "tcuts.product_category_marinades"
    
    # SKU
    if not is_raw:
        row['purchase_ok'] = "FALSE"
        row['sale_ok'] = "TRUE"
        row['use_time'] = 2
        row['alert_time'] = 1
        row['removal_time'] = 2
        row['life_time'] = 2
        row['uom_po_id/id'] = "product.product_uom_unit"
        row['uom_id/id'] = "product.product_uom_unit"
    else:
        row['purchase_ok'] = "TRUE"
        row['sale_ok'] = "FALSE"
        row['use_time'] = 2
        row['alert_time'] = 2
        row['removal_time'] = 3
        row['life_time'] = 4
        row['uom_po_id/id'] = "product.product.product_uom_kgm"
        row['uom_id/id'] = "product.product.product_uom_kgm"
    
    return row

sheet_db = sheet_db.reset_index().apply(foo, axis=1)
sheet_db = sheet_db.set_index("default_code")
sheet_db.to_csv("/Applications/MAMP/htdocs/tendercuts-site/asserts/odoo/product.template.csv")

